# Accessing Google Sheets API
Trying to access [this spreadsheet](https://docs.google.com/spreadsheets/d/1fTxHLjBa86GA7WCT-V6AbEMGRFPMJndnaVGoZZX4PMw/edit#gid=0).
[Python quickstart](https://developers.google.com/sheets/api/quickstart/python).

In [1]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 9.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 5.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.4.6
    Uninstalling google-auth-oauthlib-0.4.6:
      Successfully uninstalled google-auth-oauthlib-0.4.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.11.2 requires google-auth-oauthlib<0.5,>=0.4.1, but you have google-auth-oauthlib 1.0.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [8]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# The ID and range of a sample spreadsheet.
SPREADSHEET_ID = '1fTxHLjBa86GA7WCT-V6AbEMGRFPMJndnaVGoZZX4PMw'
RANGE_NAME = 'Anti-LGBTQ Bills!A:J'

creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            './credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

try:
    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                range=RANGE_NAME).execute()
    values = result.get('values', [])

    type(values)

    if not values:
        print('No data found.')

    print(f'Bill Information:')
    for row in values:
        # Print desired columns
        print('%s, %s, %s, %s, %s, %s' % (row[0],row[1],row[3],row[4],row[8],row[9]))
    type(values)

except HttpError as err:
    print(err)
    

Bill Information:
State, Number, Bill Type, Date, URL, Sponsors
Alaska, HB27, Trans Sports Ban, 1/19/2023, https://legiscan.com/AK/bill/HB27/2023, Thomas McKay
Alaska, SB96, Don't Say Gay/Forced Outing, 3/8/2023, https://legiscan.com/AK/bill/SB96/2023, Rules
Alaska, HB105, Don't Say Gay/Forced Outing, 3/8/2023, https://legiscan.com/AK/bill/HB105/2023, Rules
Arizona, SB1028, Drag Ban, 3/22/2023, https://legiscan.com/AZ/bill/SB1028/2023, Anthony Kern
Arizona, SB1030, Drag Ban, 3/21/2023, https://legiscan.com/AZ/bill/SB1030/2023, Anthony Kern
Arizona, SB1694, DEI Prohibitation, 3/22/2023, https://legiscan.com/AZ/bill/SB1694/2023, Jake Hoffman, Sonny Borrelli, Anthony Kern, Justine Wadsack, Justin Heap, Rachel Jones, Cory McGarr
Arizona, SB1700, Book Ban, 3/22/2023, https://legiscan.com/AZ/bill/SB1700/2023, Justine Wadsack, Rachel Jones, Cory McGarr
Arizona, HB2312, Bans trans people from working at shelters, 3/7/2023, https://legiscan.com/AZ/bill/HB2312/2023, Rachel Jones, Lupe Diaz, Liz 

IndexError: list index out of range

In [3]:
# Checking type of values and content of first row
type(values)

list

In [5]:
values[1]

['Alaska',
 'HB27',
 'Designate Sex For School-sponsored Sports',
 'Trans Sports Ban',
 '1/19/2023',
 'REFERRED TO EDUCATION',
 'Moderate',
 '',
 'https://legiscan.com/AK/bill/HB27/2023',
 'Thomas McKay']

In [6]:
# Turn values list into a dataframe
import pandas as pd
df = pd.DataFrame(values)

# Turn first row into names of columns
df.columns=df.iloc[0]

# Drop the first row
# df.iloc[row_start:row_end , column_start:column_end]
df = df.iloc[1:, :]

# Check the first few rows
df.head()

,State,Number,Summary,Bill Type,Date,Status,Erin Reed's State Risk,Notes,URL,Sponsors
0,State,Number,Summary,Bill Type,Date,Status,Erin Reed's State Risk,Notes,URL,Sponsors
1,Alaska,HB27,Designate Sex For School-sponsored Sports,Trans Sports Ban,1/19/2023,REFERRED TO EDUCATION,Moderate,,https://legiscan.com/AK/bill/HB27/2023,Thomas McKay
2,Alaska,SB96,Sex/reproduction Education; Schools,Don't Say Gay/Forced Outing,3/8/2023,REFERRED TO JUDICIARY,Moderate,In schools,https://legiscan.com/AK/bill/SB96/2023,Rules
3,Alaska,HB105,Sex/reproduction Education; Schools,Don't Say Gay/Forced Outing,3/8/2023,REFERRED TO EDUCATION,Moderate,In schools,https://legiscan.com/AK/bill/HB105/2023,Rules
4,Arizona,SB1028,Adult cabaret performances; prohibited locations,Drag Ban,3/22/2023,House read second time,Low,Bans trans performers from public.,https://legiscan.com/AZ/bill/SB1028/2023,Anthony Kern


In [9]:
# Dataframe is the correct length, same as the spreadsheet # of rows
len(df)

454

In [12]:
# Name the file with the year, month, and date.
import time
timestr = time.strftime("%Y%m%d")
print(timestr)

20230326


In [14]:
# Pickle the df so I can check against it for updates in the future.
# Should I save these in a different folder?
# Should I regularly delete these, or just save one previous pickle?

df.to_pickle(f'{timestr}-bills.pickle')
# to summon pickle
# df2 = pd.read_pickle('my_df.pickle')

In [38]:
# A function to pickle the dataframe

def datePickle(df):
    timestr = time.strftime("%Y%m%d-%H%M%S")
    df.to_pickle(f'{timestr}-df.pickle')
    print("Pickled")

# Using df as index to get bill info

In [15]:
import zipfile
import base64
import io
import glob
import time
import json
import os
import requests
import mimetypes
import csv
import pandas as pd
import numpy as np

from pypdf import PdfReader
from base64 import b64decode

from bs4 import BeautifulSoup

# U.S. state names : abbreviations dict

In [16]:
# United States of America Python Dictionary to translate States,
# Districts & Territories to Two-Letter codes and vice versa.
#
# Canonical URL: https://gist.github.com/rogerallen/1583593
#
# Dedicated to the public domain.  To the extent possible under law,
# Roger Allen has waived all copyright and related or neighboring
# rights to this code.  Data originally from Wikipedia at the url:
# https://en.wikipedia.org/wiki/ISO_3166-2:US

us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
    "US": "US"
}

In [17]:
def swapStateAbbrev(state):
    state = us_state_to_abbrev.get(state)
    return state

In [18]:
# test that it's working 
swapStateAbbrev("Wisconsin")

'WI'

## pylegiscan

To talk to LegiScan's API, we're borrowing some code from [pylegiscan](https://github.com/poliquin/pylegiscan). Since it isn't a package you can install with `pip`, it wound up being easier for distribution to just cut and paste it here.

In [55]:
# Taken from https://github.com/poliquin/pylegiscan/blob/master/pylegiscan/legiscan.py

import os
import json
import requests
from urllib.parse import urlencode
from urllib.parse import quote_plus

# current aggregate status of bill
BILL_STATUS = {1: "Introduced",
               2: "Engrossed",
               3: "Enrolled",
               4: "Passed",
               5: "Vetoed",
               6: "Failed/Dead"}

# significant steps in bill progress.
BILL_PROGRESS = {1: "Introduced",
                 2: "Engrossed",
                 3: "Enrolled",
                 4: "Passed",
                 5: "Vetoed",
                 6: "Failed/Dead",
                 7: "Veto Override",
                 8: "Chapter/Act/Statute",
                 9: "Committee Referral",
                10: "Committee Report Pass",
                11: "Committee Report DNP"}


"""
Interact with LegiScan API.

"""

# a helpful list of valid legiscan state abbreviations (no Puerto Rico)
STATES = ['ak', 'al', 'ar', 'az', 'ca', 'co', 'ct', 'dc', 'de', 'fl', 'ga',
          'hi', 'ia', 'id', 'il', 'in', 'ks', 'ky', 'la', 'ma', 'md', 'me',
          'mi', 'mn', 'mo', 'ms', 'mt', 'nc', 'nd', 'ne', 'nh', 'nj', 'nm',
          'nv', 'ny', 'oh', 'ok', 'or', 'pa', 'ri', 'sc', 'sd', 'tn', 'tx',
          'ut', 'va', 'vt', 'wa', 'wi', 'wv', 'wy']

class LegiScanError(Exception):
    pass

class LegiScan(object):
    BASE_URL = 'http://api.legiscan.com/?key={0}&op={1}&{2}'

    def __init__(self, apikey=None):
        """LegiScan API.  State parameters should always be passed as
           USPS abbreviations.  Bill numbers and abbreviations are case
           insensitive.  Register for API at http://legiscan.com/legiscan
        """
        # see if API key available as environment variable
        if apikey is None:
            apikey = config.LEGISCAN_API_KEY
        self.key = apikey.strip()

    def _url(self, operation, params=None):
        """Build a URL for querying the API."""
        if not isinstance(params, str) and params is not None:
            params = urlencode(params)
        elif params is None:
            params = ''
        return self.BASE_URL.format(self.key, operation, params)

    def _get(self, url):
        """Get and parse JSON from API for a url."""
        req = requests.get(url)
        if not req.ok:
            raise LegiScanError('Request returned {0}: {1}'\
                    .format(req.status_code, url))
        data = json.loads(req.content)
        if data['status'] == "ERROR":
            raise LegiScanError(data['alert']['message'])
        return data

    def get_session_list(self, state):
        """Get list of available sessions for a state."""
        url = self._url('getSessionList', {'state': state})
        data = self._get(url)
        return data['sessions']

    def get_dataset_list(self, state=None, year=None):
        """Get list of available datasets, with optional state and year filtering.
        """
        if state is not None:
            url = self._url('getDatasetList', {'state': state})
        elif year is not None:
            url = self._url('getDatasetList', {'year': year})
        else:
            url = self._url('getDatasetList')
        data = self._get(url)
        # return a list of the bills
        return data['datasetlist']

    def get_dataset(self, id, access_key):
        """Get list of available datasets, with optional state and year filtering.
        """
        url = self._url('getDataset', {'id': id, 'access_key': access_key})
        data = self._get(url)
        # return a list of the bills
        return data['dataset']
      
    def get_master_list(self, state=None, session_id=None):
        """Get list of bills for the current session in a state or for
           a given session identifier.
        """
        if state is not None:
            url = self._url('getMasterList', {'state': state})
        elif session_id is not None:
            url = self._url('getMasterList', {'id': session_id})
        else:
            raise ValueError('Must specify session identifier or state.')
        data = self._get(url)
        # return a list of the bills
        return [data['masterlist'][i] for i in data['masterlist']]

    def get_bill(self, bill_id=None, state=None, bill_number=None):
        """Get primary bill detail information including sponsors, committee
           references, full history, bill text, and roll call information.

           This function expects either a bill identifier or a state and bill
           number combination.  The bill identifier is preferred, and required
           for fetching bills from prior sessions.
        """
        if bill_id is not None:
            url = self._url('getBill', {'id': bill_id})
        elif state is not None and bill_number is not None:
            url = self._url('getBill', {'state': state, 'bill': bill_number})
        else:
            raise ValueError('Must specify bill_id or state and bill_number.')
        return self._get(url)['bill']

    def get_bill_text(self, doc_id):
        """Get bill text, including date, draft revision information, and
           MIME type.  Bill text is base64 encoded to allow for PDF and Word
           data transfers.
        """
        url = self._url('getBillText', {'id': doc_id})
        return self._get(url)['text']

    def get_amendment(self, amendment_id):
        """Get amendment text including date, adoption status, MIME type, and
           title/description information.  The amendment text is base64 encoded
           to allow for PDF and Word data transfer.
        """
        url = self._url('getAmendment', {'id': amendment_id})
        return self._get(url)['amendment']

    def get_supplement(self, supplement_id):
        """Get supplement text including type of supplement, date, MIME type
           and text/description information.  Supplement text is base64 encoded
           to allow for PDF and Word data transfer.
        """
        url = self._url('getSupplement', {'id': supplement_id})
        return self._get(url)['supplement']

    def get_roll_call(self, roll_call_id):
        """Roll call detail for individual votes and summary information."""
        data = self._get(self._url('getRollcall', {'id': roll_call_id}))
        return data['roll_call']

    def get_sponsor(self, people_id):
        """Sponsor information including name, role, and a followthemoney.org
           person identifier.
        """
        url = self._url('getSponsor', {'id': people_id})
        return self._get(url)['person']

    def search(self, state, bill_number=None, query=None, year=2, page=1):
        """Get a page of results for a search against the LegiScan full text
           engine; returns a paginated result set.

           Specify a bill number or a query string.  Year can be an exact year
           or a number between 1 and 4, inclusive.  These integers have the
           following meanings:
               1 = all years
               2 = current year, the default
               3 = recent years
               4 = prior years
           Page is the result set page number to return.
        """
        if bill_number is not None:
            params = {'state': state, 'bill': bill_number}
        elif query is not None:
            params = {'state': state, 'query': query,
                      'year': year, 'page': page}
        else:
            raise ValueError('Must specify bill_number or query')
        data = self._get(self._url('search', params))['searchresult']
        # return a summary of the search and the results as a dictionary
        summary = data.pop('summary')
        results = {'summary': summary, 'results': [data[i] for i in data]}
        return results

    def __str__(self):
        return '<LegiScan API {0}>'.format(self.key)

    def __repr__(self):
        return str(self)

# Connect to LegiScan

Using pylegiscan, pass your API key to `LegiScan` through an environment variable.

In [20]:
import config

api_key = config.LEGISCAN_API_KEY
legis = LegiScan(api_key)

In [21]:
# Add 'Abbreviation' column and fill with State data
df['Abbreviation'] = df.loc[:,'State']
# Check if it worked
df.head()

In [23]:
# Reorder columns and check if it worked
df = df.reindex(columns=['State','Abbreviation', 'Number', 'Summary', 'Bill Type', 'Date', 'Status', 'Erin Reed\'s State Risk', 'URL', 'Sponsors'])
df.head()

,State,Abbreviation,Number,Summary,Bill Type,Date,Status,Erin Reed's State Risk,URL,Sponsors
0,State,State,Number,Summary,Bill Type,Date,Status,Erin Reed's State Risk,URL,Sponsors
1,Alaska,Alaska,HB27,Designate Sex For School-sponsored Sports,Trans Sports Ban,1/19/2023,REFERRED TO EDUCATION,Moderate,https://legiscan.com/AK/bill/HB27/2023,Thomas McKay
2,Alaska,Alaska,SB96,Sex/reproduction Education; Schools,Don't Say Gay/Forced Outing,3/8/2023,REFERRED TO JUDICIARY,Moderate,https://legiscan.com/AK/bill/SB96/2023,Rules
3,Alaska,Alaska,HB105,Sex/reproduction Education; Schools,Don't Say Gay/Forced Outing,3/8/2023,REFERRED TO EDUCATION,Moderate,https://legiscan.com/AK/bill/HB105/2023,Rules
4,Arizona,Arizona,SB1028,Adult cabaret performances; prohibited locations,Drag Ban,3/22/2023,House read second time,Low,https://legiscan.com/AZ/bill/SB1028/2023,Anthony Kern


In [46]:
df['Abbreviation'] = df['State']

/var/folders/lj/gz81psj55xg1_fm6jfz2vbn80000gn/T/ipykernel_25117/3207276958.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Abbreviation'] = df['State']


In [48]:
# Use the swapStateAbbrev function to get all the abbreviations for the states, so we can use this to access bill info
df['Abbreviation'] = df['Abbreviation'].apply(swapStateAbbrev)

/var/folders/lj/gz81psj55xg1_fm6jfz2vbn80000gn/T/ipykernel_25117/308084209.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Abbreviation'] = df['Abbreviation'].apply(swapStateAbbrev)


In [49]:
df.head()

,State,Abbreviation,Number,Summary,Bill Type,Date,Status,Erin Reed's State Risk,URL,Sponsors,Bill ID,Bill Text
1,Alaska,AK,HB27,Designate Sex For School-sponsored Sports,Trans Sports Ban,1/19/2023,REFERRED TO EDUCATION,Moderate,https://legiscan.com/AK/bill/HB27/2023,Thomas McKay,,
2,Alaska,AK,SB96,Sex/reproduction Education; Schools,Don't Say Gay/Forced Outing,3/8/2023,REFERRED TO JUDICIARY,Moderate,https://legiscan.com/AK/bill/SB96/2023,Rules,,
3,Alaska,AK,HB105,Sex/reproduction Education; Schools,Don't Say Gay/Forced Outing,3/8/2023,REFERRED TO EDUCATION,Moderate,https://legiscan.com/AK/bill/HB105/2023,Rules,,
4,Arizona,AZ,SB1028,Adult cabaret performances; prohibited locations,Drag Ban,3/22/2023,House read second time,Low,https://legiscan.com/AZ/bill/SB1028/2023,Anthony Kern,,
5,Arizona,AZ,SB1030,Sexually explicit performances; regulation,Drag Ban,3/21/2023,House read second time,Low,https://legiscan.com/AZ/bill/SB1030/2023,Anthony Kern,,


In [50]:
datePickle(df)

Pickled


In [ ]:
# Get a warning — should have done it like the below? Anyway, it worked.
# df['Abbreviation'] = df.loc[:,'State']

df['Bill ID'] = ''
df['Bill Text'] = ''
df.head()

---
# Get the bill IDs from state data
1. Look at the state abbreviation
2. Open and load the corresponding filepath to the JSON
3. Find the bill ID
4. Add it in this df

## Get bill ID for one bill

In [53]:
stateAbbrev = df.iloc[0]['Abbreviation']
billNum = df.iloc[0]['Number']

In [60]:
stateAbbrev

'AK'

In [61]:
billNum

'HB27'

In [57]:
# Download the master list for this.
# Get list of bills for the current session in a state.
# I should just make a set from the Abbreviation column and get all the lists.
# But let's just do one for now.
masterlist = legis.get_master_list(stateAbbrev)

In [58]:
masterlist

[{'session_id': 2015,
  'state_id': 2,
  'year_start': 2023,
  'year_end': 2024,
  'prefile': 0,
  'sine_die': 0,
  'prior': 0,
  'special': 0,
  'session_tag': 'Regular Session',
  'session_title': '2023-2024 Regular Session',
  'session_name': '33rd Legislature'},
 {'bill_id': 1648141,
  'number': 'HB1',
  'change_hash': '48895fc89bb2f936b7a0c5e001155fca',
  'url': 'https://legiscan.com/AK/bill/HB1/2023',
  'status_date': '2023-01-19',
  'status': 1,
  'last_action_date': '2023-02-22',
  'last_action': 'COSPONSOR(S): PRAX',
  'title': 'Repeal Ballot Measure 2 Voting Chgs',
  'description': 'An Act relating to elections.'},
 {'bill_id': 1646412,
  'number': 'HB2',
  'change_hash': 'c3e95526966a37e6ee3919b993a1fb69',
  'url': 'https://legiscan.com/AK/bill/HB2/2023',
  'status_date': '2023-01-19',
  'status': 1,
  'last_action_date': '2023-03-17',
  'last_action': 'REFERRED TO RULES',
  'title': 'Contracts: Prohibit Israel Discrimination',
  'description': 'An Act relating to contracts 

In [62]:
type(masterlist)

list

In [65]:
type(masterlist[2])

dict

In [67]:
# Okay, what am I doing?
# I'm trying to get the description value from the dict, based on the bill number.
# And I know the state because these are all from AK.
# And I want to... add these to the DF?
# And eventually I can write a function and use df.apply or .map to just pop them in there.

In [70]:
# Check each dictionary in the masterlist for the bill number
# and print out its description, but skip the first list item
for dictionary in masterlist[1:]:
    if dictionary['number'] == billNum:
        title = dictionary['title']
        desc = dictionary['description']
        print(title)
        print(desc)
        break
    else:
        continue

Designate Sex For School-sponsored Sports
An Act relating to school athletics, recreation, athletic teams, and sports.


In [ ]:
# YES! I DON'T HAVE TO DO THE BASE64 TO PDF/HTML TO TEXT PIPELINE!
# THANK GOD
# SO MUCH EASIER AND LESS WORK-INTENSIVE FOR THE OLD PI

# Get master lists for all states in df

In [71]:
stateset = set(df['Abbreviation'])
stateset

{'AK',
 'AR',
 'AZ',
 'CO',
 'CT',
 'FL',
 'GA',
 'HI',
 'IA',
 'ID',
 'IL',
 'IN',
 'KS',
 'KY',
 'LA',
 'MD',
 'ME',
 'MI',
 'MN',
 'MO',
 'MS',
 'MT',
 'NC',
 'ND',
 'NE',
 'NH',
 'NJ',
 'NM',
 'NV',
 'OH',
 'OK',
 'OR',
 'PA',
 'RI',
 'SC',
 'SD',
 'TN',
 'TX',
 'US',
 'UT',
 'VA',
 'VT',
 'WA',
 'WI',
 'WV',
 'WY'}

In [75]:
statemasterlists = []

for state in stateset:
    statemasterlists.append(legis.get_master_list(state))

In [76]:
statemasterlists[0]

[{'session_id': 2022,
  'state_id': 15,
  'year_start': 2023,
  'year_end': 2024,
  'prefile': 0,
  'sine_die': 0,
  'prior': 0,
  'special': 0,
  'session_tag': 'Regular Session',
  'session_title': '2023-2024 Regular Session',
  'session_name': '90th General Assembly'},
 {'bill_id': 1653623,
  'number': 'HF1',
  'change_hash': '03101aec77ad879f904bd5b50599c8f5',
  'url': 'https://legiscan.com/IA/bill/HF1/2023',
  'status_date': '2023-01-11',
  'status': 1,
  'last_action_date': '2023-02-01',
  'last_action': 'Subcommittee Meeting: 02/06/2023 12:30PM RM 102, Sup. Ct. Consult.',
  'title': 'A bill for an act relating to local government funding by modifying school district funding provisions, property assessment provisions, and bond issuance requirements, and including effective date and applicability provisions.',
  'description': 'A bill for an act relating to local government funding by modifying school district funding provisions, property assessment provisions, and bond issuance r

In [77]:
statedicts = dict(zip(list(stateset),statemasterlists))

In [81]:
statedicts['AK']

[{'session_id': 2015,
  'state_id': 2,
  'year_start': 2023,
  'year_end': 2024,
  'prefile': 0,
  'sine_die': 0,
  'prior': 0,
  'special': 0,
  'session_tag': 'Regular Session',
  'session_title': '2023-2024 Regular Session',
  'session_name': '33rd Legislature'},
 {'bill_id': 1648141,
  'number': 'HB1',
  'change_hash': '48895fc89bb2f936b7a0c5e001155fca',
  'url': 'https://legiscan.com/AK/bill/HB1/2023',
  'status_date': '2023-01-19',
  'status': 1,
  'last_action_date': '2023-02-22',
  'last_action': 'COSPONSOR(S): PRAX',
  'title': 'Repeal Ballot Measure 2 Voting Chgs',
  'description': 'An Act relating to elections.'},
 {'bill_id': 1646412,
  'number': 'HB2',
  'change_hash': 'c3e95526966a37e6ee3919b993a1fb69',
  'url': 'https://legiscan.com/AK/bill/HB2/2023',
  'status_date': '2023-01-19',
  'status': 1,
  'last_action_date': '2023-03-17',
  'last_action': 'REFERRED TO RULES',
  'title': 'Contracts: Prohibit Israel Discrimination',
  'description': 'An Act relating to contracts 

# Add bill title and description to df
Change Bill ID and Bill Info columns to Bill Title and Bill Description

Get bill title and description for each bill and fill it in

In [83]:
df = df.rename(columns={'Bill ID':'Bill Title','Bill Info':'Bill Description'})

In [84]:
df.head()

,State,Abbreviation,Number,Summary,Bill Type,Date,Status,Erin Reed's State Risk,URL,Sponsors,Bill Title,Bill Text
1,Alaska,AK,HB27,Designate Sex For School-sponsored Sports,Trans Sports Ban,1/19/2023,REFERRED TO EDUCATION,Moderate,https://legiscan.com/AK/bill/HB27/2023,Thomas McKay,,
2,Alaska,AK,SB96,Sex/reproduction Education; Schools,Don't Say Gay/Forced Outing,3/8/2023,REFERRED TO JUDICIARY,Moderate,https://legiscan.com/AK/bill/SB96/2023,Rules,,
3,Alaska,AK,HB105,Sex/reproduction Education; Schools,Don't Say Gay/Forced Outing,3/8/2023,REFERRED TO EDUCATION,Moderate,https://legiscan.com/AK/bill/HB105/2023,Rules,,
4,Arizona,AZ,SB1028,Adult cabaret performances; prohibited locations,Drag Ban,3/22/2023,House read second time,Low,https://legiscan.com/AZ/bill/SB1028/2023,Anthony Kern,,
5,Arizona,AZ,SB1030,Sexually explicit performances; regulation,Drag Ban,3/21/2023,House read second time,Low,https://legiscan.com/AZ/bill/SB1030/2023,Anthony Kern,,


In [106]:
# Write the function to snag bill title and desc from the state:masterlist dict
def getBillTitle(stateAbbrev,billNum):
    masterlist = statedicts[f'{stateAbbrev}']
    # Check each dictionary in the masterlist for the bill number
    # and print out its description, but skip the first list item
    for dictionary in masterlist[1:]:
        if dictionary['number'] == billNum:
#             title = dictionary['title']
            desc = dictionary['description']
#             return title, desc
            return desc
            break
        else:
            continue

In [107]:
getBillTitle(df['Abbreviation'][1],df['Number'][1])

'An Act relating to school athletics, recreation, athletic teams, and sports.'

In [100]:
# Make a quick backup
df2=df

In [108]:
df['Bill Title'] = df.apply(lambda row: getBillTitle(row['Abbreviation'],row['Number']), axis=1)

In [109]:
df.head()

,State,Abbreviation,Number,Summary,Bill Type,Date,Status,Erin Reed's State Risk,URL,Sponsors,Bill Title,Bill Text
1,Alaska,AK,HB27,Designate Sex For School-sponsored Sports,Trans Sports Ban,1/19/2023,REFERRED TO EDUCATION,Moderate,https://legiscan.com/AK/bill/HB27/2023,Thomas McKay,"An Act relating to school athletics, recreatio...",
2,Alaska,AK,SB96,Sex/reproduction Education; Schools,Don't Say Gay/Forced Outing,3/8/2023,REFERRED TO JUDICIARY,Moderate,https://legiscan.com/AK/bill/SB96/2023,Rules,An Act relating to parental rights in a child&...,
3,Alaska,AK,HB105,Sex/reproduction Education; Schools,Don't Say Gay/Forced Outing,3/8/2023,REFERRED TO EDUCATION,Moderate,https://legiscan.com/AK/bill/HB105/2023,Rules,An Act relating to parental rights in a child&...,
4,Arizona,AZ,SB1028,Adult cabaret performances; prohibited locations,Drag Ban,3/22/2023,House read second time,Low,https://legiscan.com/AZ/bill/SB1028/2023,Anthony Kern,Adult cabaret performances; prohibited locations,
5,Arizona,AZ,SB1030,Sexually explicit performances; regulation,Drag Ban,3/21/2023,House read second time,Low,https://legiscan.com/AZ/bill/SB1030/2023,Anthony Kern,Sexually explicit performances; regulation,


In [110]:
df = df.drop(columns=['Bill Text'])

In [111]:
df.head()

,State,Abbreviation,Number,Summary,Bill Type,Date,Status,Erin Reed's State Risk,URL,Sponsors,Bill Title
1,Alaska,AK,HB27,Designate Sex For School-sponsored Sports,Trans Sports Ban,1/19/2023,REFERRED TO EDUCATION,Moderate,https://legiscan.com/AK/bill/HB27/2023,Thomas McKay,"An Act relating to school athletics, recreatio..."
2,Alaska,AK,SB96,Sex/reproduction Education; Schools,Don't Say Gay/Forced Outing,3/8/2023,REFERRED TO JUDICIARY,Moderate,https://legiscan.com/AK/bill/SB96/2023,Rules,An Act relating to parental rights in a child&...
3,Alaska,AK,HB105,Sex/reproduction Education; Schools,Don't Say Gay/Forced Outing,3/8/2023,REFERRED TO EDUCATION,Moderate,https://legiscan.com/AK/bill/HB105/2023,Rules,An Act relating to parental rights in a child&...
4,Arizona,AZ,SB1028,Adult cabaret performances; prohibited locations,Drag Ban,3/22/2023,House read second time,Low,https://legiscan.com/AZ/bill/SB1028/2023,Anthony Kern,Adult cabaret performances; prohibited locations
5,Arizona,AZ,SB1030,Sexually explicit performances; regulation,Drag Ban,3/21/2023,House read second time,Low,https://legiscan.com/AZ/bill/SB1030/2023,Anthony Kern,Sexually explicit performances; regulation


In [112]:
datePickle(df)

Pickled


# A good place to stop for the night!
Next steps:
- [ ] Create scheme for translations of bill type
- [ ] Add column to dataframe and translate
- [ ] Turn sponsor list into set
- [ ] Figure out what is necessary to run when. For example, when the Google Sheet has a new row added, look at the state and re-download the master list to get the new info and append it to the df? Check the Google Sheets API to see if there's a function for that.
- [ ] Clean up code
- [ ] Figure out Hershey Text + AxiDraw bits

In [ ]:
# For Curser, I'll need to turn everyone from the Sponsors column into a set, splitting with "," (and maybe whitespace?) and omit Rules
# I should keep track of who the last person cursed (printed) was, so they go in order